In [49]:
import requests
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import time
from time import sleep
import random
import pandas as pd
from tqdm import tqdm

In [54]:
def scroll(modal,driver):
    cnt = 0
    try:        
        # 스크롤 높이 받아오기
        last_height = driver.execute_script("return arguments[0].scrollHeight", modal)
        while cnt<100:
            cnt += 1
            pause_time =2
            # 최하단까지 스크롤
            driver.execute_script("arguments[0].scrollTo(0, arguments[0].scrollHeight);", modal)
            # 페이지 로딩 대기
            time.sleep(pause_time)
            # 무한 스크롤 동작을 위해 살짝 위로 스크롤
            driver.execute_script("arguments[0].scrollTo(0, arguments[0].scrollHeight-50);", modal)
            time.sleep(pause_time)
            # 스크롤 높이 새롭게 받아오기
            new_height = driver.execute_script("return arguments[0].scrollHeight", modal)
            try:
                # '더보기' 버튼 있을 경우 클릭
                all_review_button = driver.find_element_by_xpath('/html/body/div[1]/div[4]/c-wiz/div/div[2]/div/div/main/div/div[1]/div[2]/div[2]/div/span/span').click()
            except:
                # 스크롤 완료 경우
                if new_height == last_height:
                    print("스크롤 완료")
                    break
                last_height = new_height
                
    except Exception as e:
        print("에러 발생: ", e)

In [55]:
# URL_list = ['https://play.google.com/store/apps/details?id=com.moneeapp.mobile&hl=ko',
#             'https://play.google.com/store/apps/details?id=com.reachfree.dailyexpense&hl=ko',
#             'https://play.google.com/store/apps/details?id=com.hanabank.mzplatform&hl=ko',
URL_list=[ 'https://play.google.com/store/apps/details?id=com.kbstar.reboot&hl=ko'
           ]

In [59]:

# 크롬 드라이버 세팅
def search(URL):
    
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument('--no-sandbox')
    # chrome_options.add_argument('--disable-dev-shm-usage')
    # chrome_options.add_argument('--disable-blink-features=AutomationControlled')
    driver = webdriver.Chrome('./chromedriver', options = chrome_options)
    # 페이지 열기
    driver.get(URL)
    # 페이지 로딩 대기
    wait = WebDriverWait(driver, 5)

    # '리뷰 모두 보기' 버튼 렌더링 확인(path 수정 @2022-06-22)
    all_review_button_xpath = '/html/body/c-wiz[2]/div/div/div[1]/div[2]/div/div[1]/c-wiz[4]/section/div/div/div[5]/div/div/button/span'
    button_loading_wait = wait.until(EC.element_to_be_clickable((By.XPATH, all_review_button_xpath)))
    # '리뷰 모두 보기' 버튼 클릭
    driver.find_element(By.XPATH,all_review_button_xpath).click()

    # '리뷰 모두 보기' 페이지 렌더링 대기
    all_review_page_xpath = '/html/body/div[4]/div[2]/div/div/div/div/div[2]'
    page_loading_wait = wait.until(EC.element_to_be_clickable((By.XPATH, all_review_page_xpath)))
    time.sleep(5)
    
    # 페이지 무한 스크롤 다운
    modal = WebDriverWait(driver, 2).until(EC.element_to_be_clickable((By.XPATH, "//div[@class='fysCi']")))
    scroll(modal,driver)
    # html parsing하기
    html_source = driver.page_source
    soup_source = BeautifulSoup(html_source, 'html.parser')
    driver.quit()
    return soup_source


In [60]:
def preprocessing(soup_source):
    # 리뷰 데이터 클래스 접근
    review_source = soup_source.find_all(class_ = 'RHo1pe')
    # 리뷰 데이터 저장용 배열
    dataset = []
    # 데이터 넘버링을 위한 변수
    review_num = 0 
    # 리뷰 1개씩 접근해 정보 추출
    for review in tqdm(review_source):
        review_num+=1
        # 리뷰 등록일 데이터 추출
        date_full = review.find_all(class_ = 'bp9Aid')[0].text
        date_year = date_full[0:4] # 연도 데이터 추출
        # 해당 단어가 등장한 인덱스 추출
        year_index = date_full.find('년')
        month_index = date_full.find('월')
        day_index = date_full.find('일')

        date_month = str(int(date_full[year_index+1:month_index])) # 월(Month) 데이터 추출
        # 월 정보가 1자리의 경우 앞에 0 붙이기(e.g., 1월 -> 01월)
        if len(date_month) == 1:
            date_month = '0' + date_month

        date_day = str(int(date_full[month_index+1:day_index])) # 일(Day) 데이터 추출 
        # 일 정보가 1자리의 경우 앞에 0 붙여줌(e.g., 7일 -> 07일)
        if len(date_day) == 1:
            date_day = '0' + date_day

        # 리뷰 등록일 full version은 최종적으로 yyyymmdd 형태로 저장
        date_full = date_year + date_month + date_day
        user_name = review.find_all(class_ = 'X5PpBb')[0].text # 닉네임 데이터 추출
        rating = review.find_all(class_ = "iXRFPc")[0]['aria-label'][10] # 평점 데이터 추출
        content = review.find_all(class_ = 'h3YV2d')[0].text # 리뷰 데이터 추출

        data = {
            "id": review_num, 
            "date": date_full,
            "dateYear": date_year,
            "dateMonth": date_month,
            "dateDay": date_day,
            "rating": rating,
            "userName": user_name,
            "content": content
        }
        dataset.append(data)
    return dataset

In [61]:
for name,url in zip(['kookmin'],URL_list):
    soup_source=search(url)
    dataset = preprocessing(soup_source)
    df = pd.DataFrame(dataset)
    df.to_csv(f'{name}.csv',encoding='utf-8-sig')

/var/folders/cx/2_hdnzk567b6wtk4d2wq64yw0000gn/T/ipykernel_8562/4071868418.py:8: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./chromedriver', options = chrome_options)
/var/folders/cx/2_hdnzk567b6wtk4d2wq64yw0000gn/T/ipykernel_8562/4040251267.py:20: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  all_review_button = driver.find_element_by_xpath('/html/body/div[1]/div[4]/c-wiz/div/div[2]/div/div/main/div/div[1]/div[2]/div[2]/div/span/span').click()


스크롤 완료


100%|███████████████████████████████████████| 880/880 [00:00<00:00, 1495.00it/s]


In [47]:
dataset = preprocessing(soup_source)
df = pd.DataFrame(dataset)
df.to_csv(f'{name}.csv',encoding='utf-8-sig')

100%|████████████████████████████████████████| 520/520 [00:00<00:00, 929.37it/s]


In [62]:
df

,id,date,dateYear,dateMonth,dateDay,rating,userName,content
0,1,20221216,2022,12,16,1,이조은,좀 심각하게 많이 느려요. 솔직히 말해서 느리다기보다 그냥 오류가 났다는 상태창이 ...
1,2,20221031,2022,10,31,1,박춘,별점 0점은 없을까요? 도대체 리브서비스는 왜 사라진거죠? 왜 더 역행하는 거죠? ...
2,3,20221214,2022,12,14,5,계란비빔밥,리브에서 이용하다가 넘어왔습니다. 처음에 인증시작하는 인터페이스부터 메인화면까지 굉...
3,4,20221021,2022,10,21,1,화원,"네트워크 문제가 있다며 안열리거나 로딩이 엄청 느릴때가 비일비재 합니다. 와이파이,..."
4,5,20220806,2022,08,06,5,박서연,다양한 이벤트 덕에 계속 사용하게 되는 것 같아요! 알림도 빠르고 잔액 확인도 쉽게...
...,...,...,...,...,...,...,...,...
875,876,20220813,2022,08,13,5,PD JO,와 이런 정말 굿
876,877,20220225,2022,02,25,5,장한솔,편하고 재밌어요~~
877,878,20220722,2022,07,22,5,minseok Kim,기존 리브보다 많이 좋아진것 같습니다
878,879,20220728,2022,07,28,5,홍Star,쓰기 편한 바코드결제
